<a href="https://colab.research.google.com/github/eldor-fozilov/FracLSTM-Portfolio/blob/main/model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [222]:
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns

# Load the Data

In [45]:
selected_stocks = pd.read_csv("selected_stocks.csv", header = [0,1])
selected_stocks.head()

Unnamed: 0_level_0  Adj Close                                           \
  Unnamed: 0_level_1          A      AAPL       ABC       ABT       ADBE   
0               Date        NaN       NaN       NaN       NaN        NaN   
1         2000-01-03  43.929695  0.849469  2.980344  9.175173  16.274666   
2         2000-01-04  40.573959  0.777850  2.776868  8.913024  14.909400   
3         2000-01-05  38.057144  0.789232  2.992315  8.896641  15.204175   
4         2000-01-06  36.608093  0.720933  3.219730  9.207942  15.328290   

                                             ...     Volume             \
         ADI       ADM        ADP      ADSK  ...        WMB        WMT   
0        NaN       NaN        NaN       NaN  ...        NaN        NaN   
1  28.981533  6.548721  25.347609  8.052905  ...  2366796.0  8369900.0   
2  27.515394  6.480505  25.347609  7.660816  ...  3364720.0  6745100.0   
3  27.917074  6.378182  25.103891  7.178246  ...  2731730.0  7018700.0   
4  27.153872  6.412287  25.439016  6.740914  ...  8654732.0  6544500.0   

                                                                      \
          WY          X        XEL         XOM       XRAY        XRX   
0        NaN        NaN        NaN         NaN        NaN        NaN   
1   973700.0   494100.0  2738600.0  13458200.0   582300.0  3329164.0   
2  1201700.0  1202500.0   425200.0  14510800.0   317700.0  2361970.0   
3  1184600.0  1464500.0   500200.0  17485000.0  1188000.0  2375404.0   
4  1307700.0   675000.0   344100.0  19461600.0   534300.0  1650180.0   

                         
         YUM       ZION  
0        NaN        NaN  
1  3033493.0  1199600.0  
2  3315031.0   816100.0  
3  4642602.0  1124700.0  
4  3947658.0  1112100.0  

[5 rows x 627 columns]

In [46]:
selected_stocks.drop(0, axis = 0, inplace = True)
selected_stocks.set_index(selected_stocks.columns[0], inplace = True)
selected_stocks.index.name = 'Date'
selected_stocks.index = pd.to_datetime(selected_stocks.index)
selected_stocks

Adj Close                                                  \
                     A        AAPL         ABC         ABT        ADBE   
Date                                                                     
2000-01-03   43.929695    0.849469    2.980344    9.175173   16.274666   
2000-01-04   40.573959    0.777850    2.776868    8.913024   14.909400   
2000-01-05   38.057144    0.789232    2.992315    8.896641   15.204175   
2000-01-06   36.608093    0.720933    3.219730    9.207942   15.328290   
2000-01-07   39.658760    0.755083    3.638654    9.306244   16.072983   
...                ...         ...         ...         ...         ...   
2023-05-18  129.139999  175.050003  174.820007  108.470001  360.429993   
2023-05-19  128.869995  175.160004  173.389999  108.930000  371.250000   
2023-05-22  129.089996  174.199997  172.250000  108.250000  372.049988   
2023-05-23  128.639999  171.559998  171.259995  105.739998  370.420013   
2023-05-24  120.989998  171.839996  171.059998  104.040001  365.760010   

                                                                      ...  \
                   ADI        ADM         ADP        ADSK        AEE  ...   
Date                                                                  ...   
2000-01-03   28.981533   6.548721   25.347609    8.052905  11.028308  ...   
2000-01-04   27.515394   6.480505   25.347609    7.660816  11.028308  ...   
2000-01-05   27.917074   6.378182   25.103891    7.178246  11.454930  ...   
2000-01-06   27.153872   6.412287   25.439016    6.740914  11.412271  ...   
2000-01-07   27.917074   6.514612   26.017868    7.540173  11.582920  ...   
...                ...        ...         ...         ...        ...  ...   
2023-05-18  192.259995  72.790001  215.429993  204.839996  83.360001  ...   
2023-05-19  190.529999  73.230003  216.179993  200.889999  84.120003  ...   
2023-05-22  191.309998  73.440002  216.529999  203.490005  83.620003  ...   
2023-05-23  187.919998  75.050003  215.550003  196.639999  82.709999  ...   
2023-05-24  173.199997  72.849998  214.000000  196.750000  82.129997  ...   

               Volume                                               \
                  WMB         WMT         WY          X        XEL   
Date                                                                 
2000-01-03  2366796.0   8369900.0   973700.0   494100.0  2738600.0   
2000-01-04  3364720.0   6745100.0  1201700.0  1202500.0   425200.0   
2000-01-05  2731730.0   7018700.0  1184600.0  1464500.0   500200.0   
2000-01-06  8654732.0   6544500.0  1307700.0   675000.0   344100.0   
2000-01-07  2319625.0   7976900.0  1728000.0   904600.0   469500.0   
...               ...         ...        ...        ...        ...   
2023-05-18  6379400.0  15589500.0  4607200.0  5650900.0  2403900.0   
2023-05-19  5157200.0   9195600.0  3875500.0  5393500.0  2068500.0   
2023-05-22  5575800.0   5372600.0  2853600.0  4250100.0  1857400.0   
2023-05-23  5559400.0   5434100.0  3019900.0  5691400.0  3966300.0   
2023-05-24  5658900.0   6837900.0  4156400.0  7482300.0  3562000.0   

                                                                    
                   XOM       XRAY        XRX        YUM       ZION  
Date                                                                
2000-01-03  13458200.0   582300.0  3329164.0  3033493.0  1199600.0  
2000-01-04  14510800.0   317700.0  2361970.0  3315031.0   816100.0  
2000-01-05  17485000.0  1188000.0  2375404.0  4642602.0  1124700.0  
2000-01-06  19461600.0   534300.0  1650180.0  3947658.0  1112100.0  
2000-01-07  16603800.0  1401000.0  2366942.0  6063647.0   782000.0  
...                ...        ...        ...        ...        ...  
2023-05-18  12005600.0   938900.0   845000.0  1435400.0  9029700.0  
2023-05-19  13357700.0  1023900.0  1203300.0  1295400.0  7295900.0  
2023-05-22  12882000.0  2880800.0   915300.0  1644400.0  8414900.0  
2023-05-23  14394400.0  2093900.0  1218300.0  1828300.0  9434400.0  
2023-05-24  1633630

In [48]:
stock_returns = pd.read_csv("stock_returns.csv")
stock_returns['Date'] = pd.to_datetime(stock_returns['Date'])
stock_returns.set_index('Date', inplace = True)
stock_returns.head()

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMB,WMT,WY,X,XEL,XOM,XRAY,XRX,YUM,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,-0.079464,-0.088078,-0.070715,-0.028988,-0.087618,-0.051913,-0.010471,0.000000,-0.049914,0.000000,...,-0.022427,-0.038136,-0.038291,-0.038043,0.022765,-0.019340,0.000000,-0.047628,-0.020305,-0.049636
2000-01-05,-0.064038,0.014527,0.074724,-0.001840,0.019578,0.014493,-0.015915,-0.009662,-0.065064,0.037955,...,0.060018,-0.020620,0.051619,0.010152,0.037860,0.053082,0.017140,0.050209,0.005115,-0.001184
2000-01-06,-0.038819,-0.090514,0.073250,0.034393,0.008130,-0.027719,0.005333,0.013261,-0.062859,-0.003731,...,0.021134,0.010854,0.045717,0.006042,-0.009332,0.050405,0.001306,-0.020834,-0.008540,0.014118
2000-01-07,0.080043,0.046281,0.122316,0.010619,0.047440,0.027719,0.015832,0.022499,0.112050,0.014842,...,0.026268,0.072846,-0.033437,0.025770,0.000000,-0.002939,-0.002615,0.023408,-0.022551,0.002334
2000-01-10,0.058813,-0.017745,0.025975,-0.007067,0.037883,0.083468,0.000000,0.024292,-0.010151,-0.007394,...,-0.018693,-0.018417,-0.004369,-0.007858,0.000000,-0.014079,0.023287,-0.023408,0.039558,-0.018230


In [8]:
data_frac_adj_close = pd.read_csv('data_frac_adj_close.csv')
data_frac_adj_close

,Date,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,...,WMB,WMT,WY,X,XEL,XOM,XRAY,XRX,YUM,ZION
0,2000-01-03,43.929695,0.849469,2.980344,9.175173,16.274666,28.981533,6.548721,25.347609,8.052905,...,9.885839,43.873116,11.837615,24.076244,7.236262,18.821234,6.746934,38.057358,4.770266,37.010761
1,2000-01-04,3.508279,0.014655,0.099215,0.813067,1.050505,2.835807,1.006184,3.168451,0.992004,...,2.097748,4.528017,2.422218,23.177519,1.071154,3.315510,1.686734,36.287239,0.463132,8.906221
2,2000-01-05,0.927110,0.051627,0.361498,0.553505,1.479299,2.654308,0.511819,1.538537,0.261031,...,1.976569,2.425899,2.276460,23.414024,0.815277,3.132996,1.170845,38.155693,0.324996,6.335719
3,2000-01-06,0.706752,-0.037862,0.354702,0.716088,1.044925,0.940520,0.461891,1.567090,0.027364,...,1.392730,2.709284,1.971281,23.555927,0.336405,2.768582,0.828785,37.368973,0.176954,5.415578
4,2000-01-07,4.629581,0.050854,0.536656,0.451918,1.486837,1.987105,0.450268,1.589879,1.112478,...,1.254504,5.031893,0.842454,24.170841,0.301018,1.507430,0.644929,38.254025,0.060318,4.291654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5881,2023-05-18,4.769833,4.942911,2.762021,1.482712,14.050554,9.532783,1.637624,5.710822,12.905855,...,1.632634,5.131244,1.481835,22.170000,0.945169,4.828371,2.355033,14.620000,3.554911,3.457965
5882,2023-05-19,3.303013,2.911973,1.244161,2.297345,20.861085,3.813355,2.356284,5.113934,3.363228,...,1.460131,1.978025,1.529578,21.820000,1.223504,4.470817,1.823692,14.540000,2.361042,1.933624
5883,2023-05-22,3.656020,1.739921,1.352894,1.249358,11.895957,5.669404,2.246429,4.707007,8.658124,...,1.405006,1.920876,1.199363,21.700001,1.163638,2.698585,1.266625,14.660000,-0.904233,3.352537
5884,2023-05-23,2.995013,-0.086481,1.428677,-0.666567,8.635848,1.542907,3.675131,3.338978,-0.306516,...,1.552069,2.850093,1.288950,21.610001,0.937627,5.098812,0.520142,14.730000,-1.394339,3.644415


In [9]:
data_frac_adj_close['Date'] = pd.to_datetime(data_frac_adj_close['Date'])
data_frac_adj_close.set_index('Date', inplace = True)
data_frac_adj_close.head()

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMB,WMT,WY,X,XEL,XOM,XRAY,XRX,YUM,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,43.929695,0.849469,2.980344,9.175173,16.274666,28.981533,6.548721,25.347609,8.052905,11.028308,...,9.885839,43.873116,11.837615,24.076244,7.236262,18.821234,6.746934,38.057358,4.770266,37.010761
2000-01-04,3.508279,0.014655,0.099215,0.813067,1.050505,2.835807,1.006184,3.168451,0.992004,1.550856,...,2.097748,4.528017,2.422218,23.177519,1.071154,3.315510,1.686734,36.287239,0.463132,8.906221
2000-01-05,0.927110,0.051627,0.361498,0.553505,1.479299,2.654308,0.511819,1.538537,0.261031,1.311095,...,1.976569,2.425899,2.276460,23.414024,0.815277,3.132996,1.170845,38.155693,0.324996,6.335719
2000-01-06,0.706752,-0.037862,0.354702,0.716088,1.044925,0.940520,0.461891,1.567090,0.027364,0.648443,...,1.392730,2.709284,1.971281,23.555927,0.336405,2.768582,0.828785,37.368973,0.176954,5.415578
2000-01-07,4.629581,0.050854,0.536656,0.451918,1.486837,1.987105,0.450268,1.589879,1.112478,0.694383,...,1.254504,5.031893,0.842454,24.170841,0.301018,1.507430,0.644929,38.254025,0.060318,4.291654


In [92]:
data_volume = pd.read_csv('data_volume.csv')
data_volume['Date'] = pd.to_datetime(data_volume['Date'])
data_volume.set_index('Date', inplace = True)
data_volume.head()

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMB,WMT,WY,X,XEL,XOM,XRAY,XRX,YUM,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,4674353.0,535796800.0,2784800.0,10635087.0,7384400.0,3655600.0,984753.0,2698899.0,2845600.0,700800.0,...,2366796.0,8369900.0,973700.0,494100.0,2738600.0,13458200.0,582300.0,3329164.0,3033493.0,1199600.0
2000-01-04,4765083.0,512377600.0,944000.0,10734887.0,7813200.0,2533200.0,1088057.0,3141234.0,3348800.0,455600.0,...,3364720.0,6745100.0,1201700.0,1202500.0,425200.0,14510800.0,317700.0,2361970.0,3315031.0,816100.0
2000-01-05,5758642.0,778321600.0,810400.0,11722632.0,14927200.0,3228000.0,1087947.0,3008836.0,4297600.0,1293200.0,...,2731730.0,7018700.0,1184600.0,1464500.0,500200.0,17485000.0,1188000.0,2375404.0,4642602.0,1124700.0
2000-01-06,2534434.0,767972800.0,2695200.0,17479583.0,10221200.0,2601000.0,899971.0,2121771.0,3856800.0,320400.0,...,8654732.0,6544500.0,1307700.0,675000.0,344100.0,19461600.0,534300.0,1650180.0,3947658.0,1112100.0
2000-01-07,2819626.0,460734400.0,2543600.0,15756040.0,8253200.0,1890600.0,1186290.0,1892407.0,6387600.0,312700.0,...,2319625.0,7976900.0,1728000.0,904600.0,469500.0,16603800.0,1401000.0,2366942.0,6063647.0,782000.0


In [91]:
data_frac_order = pd.read_csv('data_frac_order.csv')
data_frac_order

,Unnamed: 0,Adj Close,Volume
0,A,0.843750,0.0
1,AAPL,0.898438,0.0
2,ABC,0.898438,0.0
3,ABT,0.882812,0.0
4,ADBE,0.851562,0.0
...,...,...,...
308,XOM,0.804688,0.0
309,XRAY,0.750000,0.0
310,XRX,0.000000,0.0
311,YUM,0.882812,0.0


In [292]:
np.corrcoef(selected_stocks[('Adj Close', 'A')], data_frac_adj_close['A'])

array([[1.      , 0.612435],
       [0.612435, 1.      ]])

# Multivariate LSTM Model

In [286]:
# Get the number of rows in the data
nrows = data.shape[0]

# Convert the data to numpy values
np_data_unscaled = np.array(data)
np_data = np.reshape(np_data_unscaled, (nrows, -1))
print(np_data.shape)

# Transform the data by standard scaling
scaler = StandardScaler()
np_data_scaled = scaler.fit_transform(np_data_unscaled)

# Creating a separate scaler that works on a single column for scaling predictions
scaler_pred = StandardScaler()
df_frac_adj_close = pd.DataFrame(data['Frac Adj Close'])
np_frac_adj_close_scaled = scaler_pred.fit_transform(df_frac_adj_close)

(5885, 2)


In [287]:
sequence_length = 42

index_frac_adj_close = data.columns.get_loc('Frac Adj Close')

# Split the training data into train and train data sets
train_data_len = math.ceil(np_data_scaled.shape[0] * 0.8)

# Create the training and test data
train_data = np_data_scaled[0:train_data_len, :]
test_data = np_data_scaled[train_data_len - sequence_length:, :]

# The LSTM needs data with the format of [samples, time steps, features]
def partition_dataset(sequence_length, data):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        x.append(data[i-sequence_length:i,:])
        y.append(data[i, index_frac_adj_close])
    
    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y

# Generate training data and test data
x_train, y_train = partition_dataset(sequence_length, train_data)
x_test, y_test = partition_dataset(sequence_length, test_data)

# Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Validate that the prediction value and the input match up
# The last close price of the second input sample should equal the first prediction value
print(x_train[1][sequence_length-1][index_frac_adj_close])
print(y_train[0])

(4666, 42, 2) (4666,)
(1177, 42, 2) (1177,)
-0.7086304442944256
-0.7086304442944256


In [339]:
# Configure the neural network model
model = Sequential()

n_neurons = 4
print(n_neurons, x_train.shape[1], x_train.shape[2])
model.add(LSTM(n_neurons, return_sequences=False, input_shape=(x_train.shape[1], x_train.shape[2]))) 
model.add(Dense(2))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

4 42 2


In [340]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 4)                 112       
                                                                 
 dense_35 (Dense)            (None, 2)                 10        
                                                                 
 dense_36 (Dense)            (None, 1)                 3         
                                                                 
Total params: 125
Trainable params: 125
Non-trainable params: 0
_________________________________________________________________


In [342]:
# Training the model
epochs = 10
batch_size = 32
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs,
                    callbacks = [early_stop]
                   )

Epoch 1/10
146/146 [==============================] - 3s 19ms/step - loss: 0.3654
Epoch 2/10
146/146 [==============================] - 2s 17ms/step - loss: 0.3635
Epoch 3/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3632
Epoch 4/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3626
Epoch 5/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3622
Epoch 6/10
146/146 [==============================] - 3s 19ms/step - loss: 0.3623
Epoch 7/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3615
Epoch 8/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3615
Epoch 9/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3605
Epoch 10/10
146/146 [==============================] - 2s 16ms/step - loss: 0.3620


In [283]:
# Get the predicted values
y_pred_scaled = model.predict(x_test)

# Unscale the predicted values
#y_pred = scaler_pred.inverse_transform(y_pred_scaled)
#y_test_unscaled = scaler_pred.inverse_transform(y_test.reshape(-1, 1))


# Mean Absolute Error (MAE)
MSE = mean_squared_error(y_test, y_pred_scaled)
print(f'Mean Squared Error: {np.round(MSE, 4)}')

# R Squared
r_squared = r2_score(y_test, y_pred_scaled)
print(f"R squared: {np.round(r_squared, 4)}")

37/37 [==============================] - 0s 7ms/step
Mean Squared Error: 0.9981
R squared: -0.0223


In [320]:
def inverse_transformation(original_data, prediction, d):
  k = 10
  w=[d]
  for i in range(2,k):
    w_ =  w[-1] * (d-i+1) / i
    w.append(w_)
  w=np.array(w[::-1]).reshape(-1,1) #sort and reshape the w
  original_prediction = prediction + np.dot(np.array(original_data), np.array(w))
  
  return original_prediction

# Long / Short Portfolio Construction